In [423]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pycountry
import pandas as pd
import os
import random

df = pd.read_csv("../../../data/unprocessed/mi/esbdata-2020-11-03-Michigan-Ingham_County.csv")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Data Structure

### Column Order

The order for column names described in the [EAVS Section B Data Standard](https://eavs-section-b-data-standard.readthedocs.io/en/latest/index.html) is Id, VoterId, RequestMethod, RequestOtherMethod, RequestDate, RequestStatusType, RequestStatusReasonType, RequestReasonOtherType, RequestType, BallotRejectionType, BallotRejectionOtherType, BallotReceivedDate, BallotReceivedMethod, BallotReceivedOtherMethod, BallotTransmissionDate, BallotTransmissionMethod, BallotTransmissionOtherMethod, BallotType, ElectionDate, ElectionName, JurisdictionId, JurisdictionIdType, JurisdictionName, StateName, VoterMailingCountry, and VoterType.

Columns in the provided dataset were changed to reflect this requirement. 

### Required Columns

Several columns (RequestOtherMethod, RequestReasonOtherType, BallotRejectionOtherType, BallotReceivedOtherMethod, and BallotTransmissionOtherMethod) were missing from the provided data and were added with empty values.

### Extra Columns

The columns labelled ADDRESS1, ADDRESS3, ADDRESS4, and ADDRESS5 are not included in the standard and so were removed. However, the data in these columns was parsed and transformed to be included in the VoterMailingCountry column.

In [424]:
standard_columns = ["Id","VoterId","RequestMethod","RequestOtherMethod","RequestDate","RequestStatusType","RequestStatusReasonType",
                    "RequestReasonOtherType","RequestType","BallotRejectionType","BallotRejectionOtherType","BallotReceivedDate",
                    "BallotReceivedMethod","BallotReceivedOtherMethod","BallotTransmissionDate","BallotTransmissionMethod",
                    "BallotTransmissionOtherMethod","BallotType","ElectionDate","ElectionName","JurisdictionId","JurisdictionIdType",
                    "JurisdictionName","StateName","VoterMailingCountry","VoterType"]

mi_columns = ["ElectionDate","ElectionName","JurisdictionId","JurisdictionIdType","JurisdictionName","StateName","VoterId","ADDRESS1",
              "ADDRESS3","ADDRESS4","ADDRESS5","VoterMailingCountry","VoterType","RequestMethod","RequestDate","RequestStatusType",
              "RequestStatusReasonType","Id","BallotTransmissionDate","RequestType","BallotType","BallotReceivedDate",
              "BallotReceivedMethod","BallotTransmissionMethod","BallotRejectionType"]

missing_columns = [column for column in standard_columns if column not in mi_columns] 
# ['RequestOtherMethod','RequestReasonOtherType',BallotRejectionOtherType',
#  'BallotReceivedOtherMethod','BallotTransmissionOtherMethod']

extra_columns = [column for column in mi_columns if column not in standard_columns]   
# ['ADDRESS1', 'ADDRESS3', 'ADDRESS4', 'ADDRESS5']


# Creating copy of ADDRESS columns to parse after dropping #

address_fields = df['ADDRESS1'].map(str) + df['ADDRESS3'].map(str) + df['ADDRESS4'].map(str) + df['ADDRESS5'].map(str)


# Add missing columns, drop nonstandard columns, and reorder #

for column in missing_columns:
    df[column] = ''

df.drop(columns=extra_columns, inplace=True)    

df = df[standard_columns]

## Column Values

### Blank Rows

Row 1015 does not appear to contain meaningful data and so was removed. Please advise if there is a reason to include it. If so, values will need to be provided for the RequestMethod, RequestStatusType, RequestType, and ElectionDate columns at minimum as these columns are required to contain some data by the standard, even if the values are all "untracked."

In [425]:
df.drop(df.index[1014], inplace=True)

### Id

Values in the Id column in the provided data included a trailing decimal and zero. This looks suspiciously like the work of Excel trying to be "helpful." These values were converted to strings and sliced to exclude these characters.

Multiple values are recorded as "n". This column requires unique values in every row. "n" values were replaced with random numerical strings as the standard accepts any unique values (for simplicity, this could even be a sequential list starting at 1).

In [426]:
df["Id"] = df["Id"].astype("str").str.slice(0,-2,1)

In [427]:
n_indeces = [787,925,995,1008,1009]

for index in n_indeces:
    df.loc[index, "Id"] = str(random.randint(10000000, 99999999))

### VoterId

As above, values in this column included a trailing decimal and zero. These values were converted to strings and sliced to exclude these characters.

In [428]:
df["VoterId"] = df["VoterId"].astype("str").str.slice(0,-2,1)

### RequestStatusReasonType

Values in this column should be recorded as arrays. Even in the case of single string values, they should be represented by arrays containing a single element (e.g., "["untracked"]" vs. "untracked"). Values were adjusted to reflect this requirement.

In [429]:
array_values = {
    'untracked': '["untracked"]'
}

df['RequestStatusReasonType'].replace(array_values, inplace=True)

### BallotRejectionType

Values in this column must be arrays. If only one value is included, it must still appear as a single-value array (e.g., "["not_timely"]" vs not_timely). Values were transformed to conform to standard.

In [430]:
array_values ={
    'not-timely': '["not-timely"]',
    'voter-moved': '["voter-moved"]',
    'missing-voter-signature': '["missing-voter-signature"]'
}

df['BallotRejectionType'].replace(array_values, inplace=True)

### ElectionName

The value provided ("2020-Presidential-Election") is valid but was changed to "2020 General Election" for consistency with other datasets.

In [431]:
df["ElectionName"] = "2020 General Election"

### VoterMailingCountry

This column was empty in the dataset provided but values were synthesized from the provided ADDRESS1, ADDRESS3, ADDRESS4, and ADDRESS5 columns.

In [432]:
df["VoterMailingCountry"] = address_fields

country_names = [country.name for country in pycountry.countries]
mi_country_names = {country: '' for country in df["VoterMailingCountry"]}


# Foreign Addresses #

for mi_country in mi_country_names:
    for country in country_names:
        if country.lower() in mi_country.lower():
            mi_country_names[mi_country] = country
            
            
# Military addresses #

# In this dataset, military addresses always include DPO, FPO, or APO so AA, AE, and AP 
# should not be incorrectly matched with longer country names

military_pos = ["APO", "DPO", "FPO"]

for mi_country in mi_country_names:
    for mil_po in military_pos:
        if mil_po in mi_country and "AA" in mi_country:
            mi_country_names[mi_country] = "Armed Forces America"
        elif mil_po in mi_country and "AE" in mi_country:
            mi_country_names[mi_country] = "Armed Forces Europe, the Middle East, Africa, and Canada"
        elif mil_po in mi_country and "AP" in mi_country:
            mi_country_names[mi_country] = "Armed Forces Pacific"
            
            
# US addresses # 

us_states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

# Avoiding mismatches when a state abbreviation is included in a 
# longer string containing a correctly named country.

country_names_lower = [country_name.lower() for country_name in country_names]

for mi_country in mi_country_names:
    for state in us_states:
        if f" {state} " in mi_country and mi_country.lower() not in country_names_lower:
            mi_country_names[mi_country] = 'United States'
            

# Account for typos, variations, and foreign spellings where possible #

corrected_country_names = {
    ' UK ': 'United Kingdom',
    'ENGLAND': 'United Kingdom',
    'GREAT BRITAIN': 'United Kingdom',
    'SCOTLAND': 'United Kingdom',
    'ITALIA': 'Italy',
    'TAIWAN': 'Taiwan, Province of China',
    'SOUTH KOREA': 'Korea, Republic of',
    'KOREA': 'Korea, Republic of',
    'ESPA�A': 'Spain'
}

for mi_country in mi_country_names:
    for corrected_country in corrected_country_names:
        if mi_country in corrected_country_names:
            mi_country_names[mi_country] = corrected_country
            
            
# Replace values #

df['VoterMailingCountry'].replace(mi_country_names, inplace=True)

### VoterType

"overseas-civilian" was changed to "overseas-citizen" to conform to the valid values of this column per the standard.

In [433]:
voter_type = {
    'overseas-civilian': 'overseas-citizen'
}

df['VoterType'].replace(voter_type, inplace=True)

In [434]:
df.to_csv(os.path.abspath(
    '../../../data/processed/mi/esbdata-2020-11-03-Michigan-Ingham_County.csv'), index=False)